### 三个项目按字母顺序排列。您需要首先按时间按升序排序，然后按支持值按降序排序，最后按项目集按字母顺序排序。如果一个月的项目集少于k个，只需按当月的顺序输出所有的项目集。

In [ ]:
from pyspark import SparkContext, SparkConf
import sys
from itertools import combinations

class Project2:   
        
    def run(self, inputPath, outputPath, k):
        conf = SparkConf().setAppName("project2_rdd").setMaster("local")
        sc = SparkContext(conf=conf)
        
        # Load the dataset
        Transactions_raw = sc.textFile(inputPath)
            
#         header = Transactions_raw.first()
#         Transactions_filtered = Transactions_raw.filter(lambda line: line != header).map(lambda line: line.split(","))
        transactions = data.map(lambda line: line.split(','))
        transactions = transactions.map(lambda fields: (fields[3].split(' ')[0], fields[1]))
        
        def extract_month_year(date):
            date_part = date.split()[0]
            month_and_year = date_part.split('/')[1] + '/' + date_part.split('/')[2]
            return month_and_year

        # Create baskets for each month
        baskets = Transactions_filtered.map(lambda fields: (extract_month_year(fields[3]), fields[1])).groupByKey().mapValues(list)
        
        # Generate combinations of 3 items in each basket
        def generate_combinations(basket):
            items = list(basket)
            item_combinations = list(combinations(items, 3))
            return item_combinations
        
        item_combinations = baskets.flatMapValues(generate_combinations)
        
        # Count the occurrences of each item set
        item_counts = item_combinations.map(lambda item: (item, 1)).reduceByKey(lambda a, b: a + b)
        
        # Calculate the total number of transactions for each month and store it in a dictionary
        total_transactions = baskets.countByKey()
        
        # Filter item sets with support >= k
        frequent_item_sets = item_counts.filter(lambda item: item[1] >= int(k))
        
        # Calculate the support for each item set
        support_values = frequent_item_sets.map(lambda item: (item[0], item[1] / total_transactions))
        
        # Format the output as specified
        #output = support_values.map(lambda item: (item[0][0], item[0][1], item[0][2], item[1])).sortBy(lambda item: (item[0], -item[3], item[0], item[2]))
        
        # Write the output to the specified output file
        support_values.coalesce(1).saveAsTextFile(outputPath)
        
        sc.stop()

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Wrong arguments")
        sys.exit(-1)
    Project2().run(sys.argv[1], sys.argv[2], sys.argv[3])


In [16]:
# 输入的数据
data = ['1', 'B', '2', '12/1/2010  8:26:00 AM', '2.3']

date_and_time = data[3]  # 获取第四个元素

# 使用空格分割日期和时间部分
date_part = date_and_time.split()[0]

# 使用斜杠分割月和年
month_and_year = date_part.split('/')[1] + '/' + date_part.split('/')[2]

print(month_and_year)

1/2010


In [12]:
date_str

'12'

In [7]:
data = ['1', 'B', '2', '12/1/2010  8:26:00 AM', '2.3']
split_data = data.split('/')
result = split_data[0].strip()
result

AttributeError: 'list' object has no attribute 'split'

In [17]:
def get_month_year(ls):
    date_and_time = ls[3]
    date_part = date_and_time.split()[0]
    month_and_year = date_part.split('/')[1] + '/' + date_part.split('/')[2]
    return month_and_year

In [18]:
get_month_year(data)

'1/2010'

In [ ]:
from pyspark import SparkContext, SparkConf
import sys
from itertools import combinations
from datetime import datetime

class Project2:   
        
    def run(self, inputPath, outputPath, k):
        conf = SparkConf().setAppName("project2_rdd").setMaster("local")
        sc = SparkContext(conf=conf)
        
        # Read the input dataset
        data = sc.textFile(inputPath)
        
        data = data.map(lambda line: line.split(','))
        
        # Define a function to extract the month/year from InvoiceDate
        def extract_month_year(date):
            date = datetime.strptime(date_str, '%d/%m/%Y %I:%M:%S %p')
            return f"{date.month}/{date.year}"
        
        # Create baskets for each month
        baskets = data.map(lambda fields: (extract_month_year(fields[3]), fields[1])).groupByKey().mapValues(list)
        
        # Generate combinations of 3 items in each basket
        def generate_combinations(basket):
            items = list(basket)
            item_combinations = list(combinations(items, 3))
            return item_combinations
        
        item_combinations = baskets.flatMapValues(generate_combinations)
        
        # Count the occurrences of each item set
        #item_counts = item_combinations.map(lambda item: (item, 1)).reduceByKey(lambda a, b: a + b)
        
        # Filter item sets with support >= k
        #frequent_item_sets = item_counts.filter(lambda item: item[1] >= int(k))
        
        # Calculate the support for each item set
        #total_transactions = baskets.countByKey()
        #support_values = frequent_item_sets.map(lambda item: (item[0][0], item[0][1], item[0][2], item[1] / total_transactions[item[0][0]]))
        
        # Format the output as specified
        #output = support_values.map(lambda item: (item[0], item[1], item[2], item[3])).sortBy(lambda item: (item[0], -item[3], item[1], item[2]))
        
        # Write the output to the specified output file
        #output.map(lambda item: f"{item[0]},({item[1]}|{item[2]}|{item[3]}),{item[4]}").coalesce(1).saveAsTextFile(outputPath)
        item_combinations.coalesce(1).saveAsTextFile(outputPath)
        sc.stop()

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Wrong arguments")
        sys.exit(-1)
    Project2().run(sys.argv[1], sys.argv[2], sys.argv[3])

Problem Definition: You are given an E-Commerce dataset of customer purchase transaction logs collected over time. Each record in the dataset has the following five fields (see the example dataset): 

InvoiceNo: the unique ID to record one purchase transaction
Description: the name of the item in a transaction
Quantity: the amount of the items purchased
InvoiceDate: the time of the transaction
UnitPrice: the price of a single item

Your task is to utilize Spark to detect the top-k frequent item sets from the log for each month. To make the problem simple, you are only required to find frequent item sets containing three items. The support of an item set X in a month M is computed as: 

Support(X) = (Number of transactions containing X in M) / (Total number of transactions in M)

In [22]:
from datetime import datetime
def extract_month_year(date_str):
        date = datetime.strptime(date_str, '%d/%m/%Y %I:%M:%S %p')
        return f"{date.month}/{date.year}"
    
extract_month_year('12/1/2010  8:26:00 AM')

'1/2010'

In [20]:
extract_month_year('12/1/2010  8:26:00 AM')

ValueError: unconverted data remains: :00 AM

In [ ]:
from pyspark import SparkContext, SparkConf
import sys

class Project2:   
        
    def run(self, inputPath, outputPath, k):
        conf = SparkConf().setAppName("project2_rdd").setMaster("local")
        sc = SparkContext(conf=conf)
        
        # Load the dataset
        Transactions_raw = sc.textFile(inputPath)
        
        # Split the words in csv file, return a list 
        transactions_splitted = Transactions_raw.map(lambda line: line.split(','))
        
        # The function for extracting the month and year from the fourth element
        def extract_month_year(ls):
            date_and_time = ls[3]
            date_part = date_and_time.split()[0]
            month_and_year = date_part.split('/')[1] + '/' + date_part.split('/')[2]
            ls[3] = month_and_year
            return ls
        
        # Extract the month and year 
        transactions_month_year = transactions_splitted.map(lambda data: extract_month_year(data))
        
        transactions = transactions_month_year.map(lambda x: (x[0],(x[1],x[3])))
        
        total_transactions = transactions.countByKey()

        total_transactions.coalesce(1).saveAsTextFile(outputPath)
        
        sc.stop()

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Wrong arguments")
        sys.exit(-1)
    Project2().run(sys.argv[1], sys.argv[2], sys.argv[3])


In [1]:
original_list = ['1', 'A', '3', '12/1/2010  8:26:00 AM', '2.5']

# 切割第四个元素并提取需要的部分
date_time = original_list[3].split()
date = date_time[0]  # 提取日期部分

# 修改第四个元素
original_list[3] = date

# 打印修改后的列表
print(original_list)

['1', 'A', '3', '12/1/2010', '2.5']


In [2]:
def extract_month_year(ls):
    date_and_time = ls[3]
    date_part = date_and_time.split()[0]
    month_and_year = date_part.split('/')[1] + '/' + date_part.split('/')[2]
    ls[3] = month_and_year
    return ls

In [3]:
extract_month_year(original_list)

['1', 'A', '3', '1/2010', '2.5']

In [2]:
from itertools import combinations
list(combinations(range(4), 3))

[(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]

In [ ]:
def generate_combinations(basket):
    items = list(basket)
    item_combinations = list(combinations(items, 3))
    return item_combinations

In [ ]:
basket = 
generate_combinations()

# RDD格式：

In [ ]:
# spark-submit project2_rdd.py "file:///home/sample.csv" "file:///home/output" 2
# spark-submit project2_rdd.py "file:///home/test.csv" "file:///home/output_rdd" 5

In [ ]:
from pyspark import SparkContext, SparkConf
import sys
from itertools import combinations
from datetime import datetime

def extract_month_year(date):
    date = date.split(' ')[0]
    split_date = date.split("/")
    new_date = "/".join([split_date[1], split_date[2]])
    return new_date

def generate_combinations(basket):
    items = list(basket)
    unique_tuples = set()
    new_tuples = []
    item_combinations = list(combinations(items, 3))
    for item in item_combinations:
        if len(set(item)) == len(item):
            unique_tuples.add(item)
    unique_tuples_list = list(unique_tuples)
    sorted_data = [tuple(sorted(item)) for item in unique_tuples_list]
    unique_sorted_data = list(dict.fromkeys(sorted_data))
    for tup in unique_sorted_data:
        new_tup = '|'.join(tup)
        new_tuples.append(f'({new_tup})')
    return new_tuples

def sort_function(row):
    return -row[1][1]

class Project2:

    def run(self, inputPath, outputPath, k):
        conf = SparkConf().setAppName("project2_rdd").setMaster("local")
        sc = SparkContext(conf=conf)

        # Read the input CSV file
        data = sc.textFile(inputPath)

        # Split the words in csv file, return a list 
        data = data.map(lambda line: line.split(','))

        # Extract month and year from the InvoiceDate field and make it with Invoice No. as the key
        Invoice_Description_Month_Year = data.map(lambda fields: ((extract_month_year(fields[3]),fields[0]),fields[1]))
        
        # Generate all possible combinations of three items for each transaction
        Items_GroupedByInvoiceAndTime = Invoice_Description_Month_Year.groupByKey().map(lambda x: (x[0],generate_combinations(x[1])))

        # Calculate the total number of transactions 
        total_transactions = Items_GroupedByInvoiceAndTime.map(lambda x: (x[0][0], x[0][1])) \
                                                          .distinct() \
                                                          .map(lambda x: (x[0], 1)) \
                                                          .reduceByKey(lambda x, y: x + y)
        
        # Using flatMap to make the elements flatten to get a new key-value
        Items_Splitted = Items_GroupedByInvoiceAndTime.flatMap(lambda x: [(x[0][0], item) for item in x[1]])

        # Using map to count every key-value
        counts = Items_Splitted.map(lambda x: (x[0], x[1], 1))
        
        # Count the occurrences of each item set according to the date
        Total_counts_By_Date_Items = counts.map(lambda x: ((x[0], x[1]), x[2])).reduceByKey(lambda x, y: x + y)

        # Calculate the support for each item set in each month
        supports = Total_counts_By_Date_Items.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                                            .join(total_transactions) \
                                            .map(lambda x: (x[0], (x[1][0][0], x[1][0][1] / x[1][1])))
        
        # Sort the results by time and support values
        Sorted_Date_Supports = supports.sortBy(lambda x: (x[0], -x[1][1], x[1][0]))
        
        # Group the data by date
        Grouped_result = Sorted_Date_Supports.groupBy(lambda x: x[0])

        # Sort monthly data and use mapValues function to get top k data in each month
        top_k_transactions = Grouped_result.flatMap(lambda x: sorted(list(x[1]), key = sort_function)[:int(k)])

        # Re-sort the transaction by specified rules
        result = top_k_transactions.sortBy(lambda x: (datetime.strptime(x[0], "%m/%Y"), -x[1][1], x[1][0]))

        # Format the final result
        final_result = result.map(lambda x: f'{x[0]},{x[1][0]},{x[1][1]}')
        
        # Output the final result in one file
        final_result.coalesce(1).saveAsTextFile(outputPath)

        sc.stop()

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Wrong arguments")
        sys.exit(-1)
    Project2().run(sys.argv[1], sys.argv[2], sys.argv[3])

# DF格式

In [ ]:
# spark-submit project2_df.py "file:///home/sample.csv" "file:///home/output" 2
# spark-submit project2_df.py "file:///home/test.csv" "file:///home/output_df" 5

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from pyspark.sql.window import Window
from itertools import combinations

def generate_combinations(basket):
    items = list(basket)
    unique_tuples = set()
    new_tuples = []
    item_combinations = list(combinations(items, 3))
    for item in item_combinations:
        if len(set(item)) == len(item):
            unique_tuples.add(item)
    unique_tuples_list = list(unique_tuples)
    sorted_data = [tuple(sorted(item)) for item in unique_tuples_list]
    unique_sorted_data = list(dict.fromkeys(sorted_data))
    for tup in unique_sorted_data:
        new_tup = '|'.join(tup)
        new_tuples.append(f'({new_tup})')
    return new_tuples

class Project2:           
    def run(self, inputPath, outputPath, k):
        spark = SparkSession.builder.master("local").appName("project2_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputPath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("InvoiceDate",  split(data["InvoiceDate"], ' ')[0])
        data_filtered = data_filtered.withColumn("InvoiceDate", concat_ws("/", split(col("InvoiceDate"), "/")[1], split(col("InvoiceDate"), "/")[2]))

        # Make the InvoiceNo and InvoiceDate as the key to get all the items
        Description_combinations = data_filtered.groupBy("InvoiceNo", "InvoiceDate").agg(collect_list("Description").alias("Descriptions"))
        
        # Combine all the items according to their keys and each one is string type in a list
        Description_combinations_udf = udf(generate_combinations, StringType())
        data_combined = Description_combinations.withColumn("Descriptions", Description_combinations_udf(Description_combinations["Descriptions"]))
        
        # Calculate the total number of transactions 
        total_transactions = data_combined.groupBy("InvoiceDate").count()

        # Split the items list and get the unique tuple of items
        Descriptions_splitted = data_combined.withColumn("Descriptions", split(col("Descriptions"), ", "))
        Descriptions_flatten = Descriptions_splitted.withColumn("Descriptions", explode(Descriptions_splitted["Descriptions"]))
        Descriptions_formatted = Descriptions_flatten.withColumn("Descriptions", regexp_replace(col("Descriptions"), r"[\[\]]", ""))
        Descriptions_formatted = Descriptions_formatted.filter(col("Descriptions")!= "")

        # Drop the column InvoiceNo and count the number of item tuples
        Date_Descriptions = Descriptions_formatted.drop("InvoiceNo")
        Date_Descriptions_Counts = Date_Descriptions.groupBy("InvoiceDate", "Descriptions").agg(count("*").alias("Counts"))

        # Merge the number of total transactions and the number of item tuples 
        Date_Descriptions_double = Date_Descriptions_Counts.join(total_transactions, "InvoiceDate", "inner")

        # Calculate the support values
        Supports_cal = Date_Descriptions_double.withColumn("Support value", col("Counts") / col("count"))
        summary = Supports_cal.select("InvoiceDate", "Descriptions", "Support value")

        # To order the date by itself, here transform the string type of InvoiceDate to date type
        summary = summary.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "M/y"))
        # Sort the results first by time in ascending order, then by the support value in descending order, and finally by the item set in alphabetical order
        result = summary.orderBy(col("InvoiceDate").asc(), col("Support value").desc(), col("Descriptions").asc())

        # Define a window for get top k
        window_specification = Window.partitionBy("InvoiceDate").orderBy(col("Support value").desc())
        final_result = result.withColumn("row_num", row_number().over(window_specification)).filter(col("row_num") <= int(k)).drop("row_num")
        
        # Transform the date type of InvoiceDate to the original string type
        final_result = final_result.withColumn("InvoiceDate", date_format(col("InvoiceDate"), "M/y"))
        
        # Write the final results to a csv file
        final_result.write.format("csv").option("header", "false").save(outputPath)
        
        # final_result.show()
        spark.stop()

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Wrong arguments")
        sys.exit(-1)
    Project2().run(sys.argv[1], sys.argv[2], sys.argv[3])


###  ----------------------------------------------------------- Test -----------------------------------------------------------

In [ ]:
datetime.strptime(x[0], "%m/%Y")

In [11]:
def process_row(row):
    row = row.split(' ')[0]
    split_date = row.split("/")
    new_date = "/".join([split_date[1], split_date[2]])
    return new_date

In [12]:
a = '12/1/2010  8:26:00 AM'
process_row(a)

'1/2010'

In [10]:
a.split(' ')[0]

'12/1/2010'

In [13]:
def custom_time_sort(time_data):
    # 将时间数据转换为元组列表，每个元组包含年份和月份
    time_tuples = [(int(month.split('/')[0]), int(month.split('/')[1])) for month in time_data]

    # 使用自定义排序规则，首先按年份排序，然后按月份排序
    sorted_time_tuples = sorted(time_tuples, key=lambda x: (x[0], x[1]))

    # 将排序后的元组列表转换回原始时间格式
    sorted_time_data = ['{}/{}'.format(month[0], month[1]) for month in sorted_time_tuples]

    return sorted_time_data

# 测试
time_data = ["1/2010", "2/2010", "3/2010", "4/2010", "5/2010", "6/2010", "7/2010", "8/2010", "9/2010", "10/2010", "11/2010", "12/2010", "1/2011", "2/2011", "3/2011"]
sorted_time_data = custom_time_sort(time_data)
print(sorted_time_data)


['1/2010', '1/2011', '2/2010', '2/2011', '3/2010', '3/2011', '4/2010', '5/2010', '6/2010', '7/2010', '8/2010', '9/2010', '10/2010', '11/2010', '12/2010']


In [14]:
def custom_time_sort(time_data):
    def time_to_number(time_str):
        month, year = map(int, time_str.split('/'))
        return year * 12 + month
    sorted_time_data = sorted(time_data, key=time_to_number)
    return sorted_time_data

# 测试
time_data = ["1/2010", "2/2010", "3/2010", "4/2010", "5/2010", "6/2010", "7/2010", "8/2010", "9/2010", "10/2010", "11/2010", "12/2010", "1/2011", "2/2011", "3/2011"]
sorted_time_data = custom_time_sort(time_data)
print(sorted_time_data)


['1/2010', '2/2010', '3/2010', '4/2010', '5/2010', '6/2010', '7/2010', '8/2010', '9/2010', '10/2010', '11/2010', '12/2010', '1/2011', '2/2011', '3/2011']


In [ ]:
date_part_rdd = rdd.map(lambda line: line.split(',')[3].split()[0].split('/')[1] + '/' + line.split(',')[3].split()[0].split('/')[2])

In [1]:
from datetime import datetime
a = '1/2020'
datetime.strptime(a,'%m/%Y')

datetime.datetime(2020, 1, 1, 0, 0)

In [2]:
datetime.strptime(a, '%m/%Y').date()

datetime.date(2020, 1, 1)

In [ ]:
# Sort the data by date in ascending order and support value in descending order
result_rdd = rdd.sortBy(lambda x: (x[0], -x[1][1]))

# Group the data by date
grouped_rdd = result_rdd.groupBy(lambda x: x[0])

# Define k
k = 2

# Take the top k items for each month
result_rdd = grouped_rdd.flatMap(lambda x: x[1][:k])

# Format the result
result_rdd = result_rdd.map(lambda x: f'{x[0]},{x[1][0]},{x[1][1]}')

In [ ]:
def Question1(self, fields):
    pairs = fields.map(lambda x: (x[VoteTypeID], set([x[PostId]])))

    res = pairs.reduceByKey(lambda x, y: x | y)
    res = res.map(lambda x: (x[0], len(x[1]))).sortByKey().sortBy(lambda x: x[1], ascending =False)
    res = res.take(5)
    for r in res:
        print(r) 

In [1]:
from datetime import datetime

def sort_dates_by_year_month(dates):
    def custom_sort(date_str):
        date = datetime.strptime(date_str, "%m/%Y")
        return (date.year, date.month)
    
    return sorted(dates, key=custom_sort)

# Example usage:
dates = ["03/2022", "10/2021", "07/2023", "01/2022", "12/2021", "04/2022"]
sorted_dates = sort_dates_by_year_month(dates)
print(sorted_dates)


['10/2021', '12/2021', '01/2022', '03/2022', '04/2022', '07/2023']


In [2]:
date_str = '2/2010'

datetime.strptime(date_str, "%m/%Y")

datetime.datetime(2010, 2, 1, 0, 0)

In [3]:
from datetime import datetime

# 定义时间字符串列表
time_strings = ["1/2010", "2/2010", "3/2010", "1/2011", "2/2011"]

# 将时间字符串转换为日期对象并排序
date_objects = [datetime.strptime(time_str, "%m/%Y") for time_str in time_strings]
sorted_dates = sorted(date_objects)

# 打印排序后的日期
for date in sorted_dates:
    print(date.strftime("%m/%Y"))


01/2010
02/2010
03/2010
01/2011
02/2011


In [4]:
sorted_dates

[datetime.datetime(2010, 1, 1, 0, 0),
 datetime.datetime(2010, 2, 1, 0, 0),
 datetime.datetime(2010, 3, 1, 0, 0),
 datetime.datetime(2011, 1, 1, 0, 0),
 datetime.datetime(2011, 2, 1, 0, 0)]

In [5]:
from datetime import datetime

# 定义时间字符串列表
time_strings = ["1/2010", "2/2010", "3/2010", "1/2011", "2/2011"]

# 定义一个函数来将时间字符串转换为日期对象
def parse_date_string(date_string):
    # 这里假设时间字符串的格式是：月/年
    month, year = map(int, date_string.split('/'))
    return datetime(year, month, 1)

# 使用parse_date_string函数来将时间字符串列表转换为日期对象列表
date_objects = [parse_date_string(time_str) for time_str in time_strings]

# 对日期对象列表进行排序
sorted_date_objects = sorted(date_objects)

# 打印排序后的日期对象
for date in sorted_date_objects:
    print(date.strftime("%m/%Y"))


01/2010
02/2010
03/2010
01/2011
02/2011


In [6]:
from datetime import datetime

# 定义时间字符串列表
time_strings = ["1/2010", "2/2010", "3/2010", "1/2011", "2/2011"]

# 定义一个函数来将时间字符串转换为日期对象
def parse_date_string(date_string):
    # 这里假设时间字符串的格式是：月/年
    month, year = map(int, date_string.split('/'))
    return datetime(year, month, 1)

# 使用parse_date_string函数来将时间字符串列表转换为日期对象列表
date_objects = [parse_date_string(time_str) for time_str in time_strings]

# 对日期对象列表进行排序
sorted_date_objects = sorted(date_objects)

# 打印排序后的日期对象
for date in sorted_date_objects:
    print(date.strftime("%m/%Y"))


01/2010
02/2010
03/2010
01/2011
02/2011


In [4]:
data = [('A', 'B', 'C'),
        ('A', 'B', 'A'),
        ('A', 'B', 'C'),
        # 这里包括其他所有的元组...
        ('A', 'B', 'C')]

# 创建一个集合用于存储不重复的元组
unique_tuples = set()

# 遍历原始元组列表，将不重复的元组添加到集合中
for item in data:
    if len(set(item)) == len(item):
        unique_tuples.add(item)

# 转换集合为列表
unique_tuples_list = list(unique_tuples)

# 打印结果
for item in unique_tuples_list:
    print(item)

('A', 'B', 'C')


In [13]:
def generate_combinations(basket):
    items = list(basket)
    unique_tuples = set()
    item_combinations = list(combinations(items, 3))
    for item in item_combinations:
        if len(set(item)) == len(item):
            unique_tuples.add(item)
    unique_tuples_list = list(unique_tuples)
    sorted_data = [tuple(sorted(item)) for item in unique_tuples_list]
    unique_sorted_data = list(dict.fromkeys(sorted_data))
    return unique_sorted_data

In [14]:
unique_tuples_list

[('A', 'B', 'C')]

In [9]:
# 原始元组列表
data = [('C', 'D', 'A'), ('C', 'A', 'B'), ('A', 'B', 'C'), ('A', 'C', 'B'), ('D', 'A', 'B'),
        ('B', 'D', 'C'), ('D', 'A', 'C'), ('B', 'C', 'D'), ('B', 'A', 'D'), ('C', 'D', 'B'),
        ('B', 'C', 'A'), ('C', 'B', 'D'), ('A', 'D', 'B'), ('A', 'B', 'D'), ('A', 'D', 'C'),
        ('A', 'C', 'D'), ('B', 'D', 'A'), ('C', 'A', 'D'), ('B', 'A', 'C'), ('D', 'B', 'C'),
        ('A', 'B', 'C')]

# 对每个元组排序
sorted_data = [tuple(sorted(item)) for item in data]

# 去除重复项并保持原始顺序
unique_sorted_data = list(dict.fromkeys(sorted_data))

# 打印结果
for item in unique_sorted_data:
    print(item)


('A', 'C', 'D')
('A', 'B', 'C')
('A', 'B', 'D')
('B', 'C', 'D')


In [15]:
generate_combinations(data)

[(('C', 'A', 'B'), ('B', 'D', 'C'), ('C', 'B', 'D')),
 (('B', 'D', 'C'), ('A', 'D', 'C'), ('B', 'D', 'A')),
 (('B', 'C', 'D'), ('A', 'D', 'C'), ('A', 'C', 'D')),
 (('B', 'D', 'A'), ('C', 'A', 'D'), ('B', 'A', 'C')),
 (('D', 'A', 'B'), ('A', 'B', 'D'), ('A', 'B', 'C')),
 (('A', 'C', 'B'), ('A', 'D', 'C'), ('C', 'A', 'D')),
 (('C', 'A', 'B'), ('B', 'A', 'D'), ('C', 'D', 'B')),
 (('B', 'C', 'A'), ('B', 'A', 'C'), ('D', 'B', 'C')),
 (('C', 'A', 'B'), ('B', 'A', 'D'), ('A', 'B', 'C')),
 (('C', 'A', 'B'), ('C', 'B', 'D'), ('A', 'D', 'B')),
 (('A', 'B', 'C'), ('C', 'D', 'B'), ('D', 'B', 'C')),
 (('D', 'A', 'C'), ('A', 'D', 'B'), ('B', 'A', 'C')),
 (('B', 'C', 'D'), ('A', 'B', 'D'), ('A', 'D', 'C')),
 (('A', 'D', 'C'), ('B', 'D', 'A'), ('B', 'A', 'C')),
 (('B', 'C', 'A'), ('B', 'D', 'A'), ('C', 'A', 'D')),
 (('D', 'A', 'C'), ('C', 'B', 'D'), ('A', 'B', 'D')),
 (('C', 'A', 'B'), ('B', 'D', 'C'), ('A', 'C', 'D')),
 (('A', 'B', 'C'), ('C', 'A', 'D'), ('D', 'B', 'C')),
 (('B', 'D', 'C'), ('B', 'C'

In [4]:
tuples = [('A', 'C', 'D'), ('A', 'B', 'C'), ('A', 'B', 'D'), ('B', 'C', 'D')]
new_tuples = []

for tup in tuples:
    new_tup = '|'.join(tup)
    new_tuples.append(f'({new_tup})')

print(new_tuples)


['(A|C|D)', '(A|B|C)', '(A|B|D)', '(B|C|D)']


In [1]:
# 给定的元组列表
tuples = [(('A', 'C', 'D'), 1), (('A', 'B', 'C'), 2), (('A', 'B', 'D'), 1), (('B', 'C', 'D'), 1)]

# 创建一个字典，用于将相同数字的元组合并为一个列表
result_dict = {}

for tup, num in tuples:
    key = tuple(sorted(tup))  # 使用排序的元组作为键
    if key in result_dict:
        result_dict[key].append(num)
    else:
        result_dict[key] = [num]

# 创建一个新的元组列表，包含字母表示形式和相同数字的合并值
new_tuples = [(f"({'|'.join(key)})", sum(values)) for key, values in result_dict.items()]

# 打印新的元组列表
for tup in new_tuples:
    print(tup)

('(A|C|D)', 1)
('(A|B|C)', 2)
('(A|B|D)', 1)
('(B|C|D)', 1)


In [5]:
def generate_combinations(basket):
    items = list(basket)
    unique_tuples = set()
    new_tuples = []
    item_combinations = list(combinations(items, 3))
    for item in item_combinations:
        if len(set(item)) == len(item):
            unique_tuples.add(item)
    unique_tuples_list = list(unique_tuples)
    sorted_data = [tuple(sorted(item)) for item in unique_tuples_list]
    unique_sorted_data = list(dict.fromkeys(sorted_data))
    for tup in unique_sorted_data:
        new_tup = '|'.join(tup)
        new_tuples.append(f'({new_tup})')
    return new_tuples

In [7]:
data = [
    ('1/2010', '1', ['(A|B|C)']),
    ('1/2010', '2', ['(A|C|D)']),
    ('1/2010', '3', ['(A|C|D)', '(B|C|D)', '(A|B|D)', '(A|B|C)']),
    ('2/2010', '4', ['(A|B|C)'])
]

# 使用列表推导式将数据转换为所需的格式
result = [(entry[0], entry[1], item, 1) for entry in data for item in entry[2]]

# 打印转换后的结果
for item in result:
    print(item)


('1/2010', '1', '(A|B|C)', 1)
('1/2010', '2', '(A|C|D)', 1)
('1/2010', '3', '(A|C|D)', 1)
('1/2010', '3', '(B|C|D)', 1)
('1/2010', '3', '(A|B|D)', 1)
('1/2010', '3', '(A|B|C)', 1)
('2/2010', '4', '(A|B|C)', 1)


In [9]:
# 原始数据
data = [
    ('1/2010', '1', ['(A|B|C)']),
    ('1/2010', '2', ['(A|C|D)']),
    ('1/2010', '3', ['(A|C|D)', '(B|C|D)', '(A|B|D)', '(A|B|C)']),
    ('2/2010', '4', ['(A|B|C)'])
]

# 转换数据
transformed_data = []
for key, subkey, values in data:
    for value in values:
        transformed_data.append(((key, subkey, value), 1))

# 输出结果
for item in transformed_data:
    print(item)


(('1/2010', '1', '(A|B|C)'), 1)
(('1/2010', '2', '(A|C|D)'), 1)
(('1/2010', '3', '(A|C|D)'), 1)
(('1/2010', '3', '(B|C|D)'), 1)
(('1/2010', '3', '(A|B|D)'), 1)
(('1/2010', '3', '(A|B|C)'), 1)
(('2/2010', '4', '(A|B|C)'), 1)


In [14]:
def transformed_counts(ls):
    data = ls
    transformed_data = []
    for key, subkey, values in data:
        for value in values:
            transformed_data.append(((key, subkey, value), 1))
    return transformed_data

In [15]:
transformed_counts(data)

[(('1/2010', '1', '(A|B|C)'), 1),
 (('1/2010', '2', '(A|C|D)'), 1),
 (('1/2010', '3', '(A|C|D)'), 1),
 (('1/2010', '3', '(B|C|D)'), 1),
 (('1/2010', '3', '(A|B|D)'), 1),
 (('1/2010', '3', '(A|B|C)'), 1),
 (('2/2010', '4', '(A|B|C)'), 1)]

In [16]:
input_data = [
    ('1/2010', '1', ['(A|B|C)']),
    ('1/2010', '2', ['(A|C|D)']),
    ('1/2010', '3', ['(A|C|D)', '(B|C|D)', '(A|B|D)', '(A|B|C)']),
    ('2/2010', '4', ['(A|B|C)'])
]

# 定义一个空列表来存储转换后的数据
output_data = []

# 遍历输入数据并进行转换
for date, pattern, patterns_list in input_data:
    for pattern_str in patterns_list:
        output_data.append((date, pattern_str, 1))

# 打印转换后的数据
for item in output_data:
    print(item)


('1/2010', '(A|B|C)', 1)
('1/2010', '(A|C|D)', 1)
('1/2010', '(A|C|D)', 1)
('1/2010', '(B|C|D)', 1)
('1/2010', '(A|B|D)', 1)
('1/2010', '(A|B|C)', 1)
('2/2010', '(A|B|C)', 1)


In [17]:
def transformed_counts(ls):
    data = ls
    output_data = []
    for key, values in input_data:
        for value in values:
            output_data.append((key[0], value, 1))
    return output_data

In [18]:
transformed_counts(input_data)

ValueError: too many values to unpack (expected 2)